In [1]:
# Importando bibliotecas necessárias

import pandas as pd
import os
import glob
import numpy as np
from datetime import datetime

In [2]:
# Definindo informações importantes

loja = 1
periodo = 2

In [3]:
ficha_3 = pd.read_excel(f'C:\Bots\Bot_transmissao\Geracao CAT\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_v1.xlsx') 
ficha_3

,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,COD_ITEM,CNPJ EMITENTE,tipo,vBCST,FONTE
0,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,1,1.0,23.422914,...,0.00,0.000000,0.0,0.0,0.000000,100073,NaN,grupo,0.000000,dfe
1,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,1,NaN,NaN,...,63.62,71.877277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
2,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,1,NaN,NaN,...,63.62,71.877277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
3,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,1,NaN,NaN,...,0.00,0.000000,0.0,0.0,0.000000,100073,NaN,grupo,0.000000,dfe
4,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,1,NaN,NaN,...,63.72,71.777277,0.0,0.0,24.389510,100073,2.784996e+13,grupo,75.582468,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,1,NaN,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe
493478,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,1,NaN,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe
493479,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,1,NaN,NaN,...,1.88,8.606188,0.0,0.0,2.621547,99234,2.784996e+13,grupo,5.632534,entrada_sp
493480,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,1,NaN,NaN,...,0.00,0.000000,0.0,0.0,0.000000,99234,NaN,grupo,0.000000,dfe


In [3]:
def recalcular_digito_verificador(nfe_completa):
    # Verifica se a NF-e com o dígito verificador tem o tamanho correto
    if pd.isnull(nfe_completa) or len(nfe_completa) != 44:
        raise ValueError(f"Valor inválido na coluna 'CHV_DOC': {nfe_completa}")

    # Remove o último caractere (dígito verificador)
    nfe_sem_dv = nfe_completa[:-1]

    # Restante da lógica de cálculo do dígito verificador
    soma = 0
    peso = 2
    nfe_invertida = nfe_sem_dv[::-1]

    for digito in nfe_invertida:
        soma += int(digito) * peso
        peso += 1
        if peso > 9:
            peso = 2

    resto = soma % 11
    dv = 0 if resto == 0 or resto == 1 else 11 - resto

    return dv

df_0000 = pd.read_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\0000_0150_P{periodo}.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000 = df_0000[df_0000['LOJA'] == f'Loja {loja}']


cnpj_loja = df_0000[df_0000['LOJA'] == f'Loja {loja}']['CNPJ'].values[0]

# ficha_3['CHV_DOC'] = np.where(ficha_3['FONTE'].isin(['entrada_oprprp', 'saida_nfe', 'dfe']),
#                            ficha_3['CHV_DOC'].str.slice(0,6) + cnpj_loja + ficha_3['CHV_DOC'].str.slice(20,44),
#                            ficha_3['CHV_DOC'])
    
# ficha_3['DV_NFe'] = ficha_3['CHV_DOC'].apply(recalcular_digito_verificador)
# ficha_3['CHV_DOC'] = ficha_3['CHV_DOC'].str.slice(0,-1) + ficha_3['DV_NFe'].astype(str)
# ficha_3

In [5]:
# Lista todos os arquivos TXT na pasta
pasta_arquivos = f'C:\Bots\Bot_transmissao\Geracao CAT\Loja_{loja}\Teste_loja_{loja}'

# Filtra apenas os arquivos que começam com '1100'
arquivos_1100 = [arquivo for arquivo in os.listdir(pasta_arquivos) if arquivo.startswith('1100')]
arquivos_1100
# Lista para armazenar os dataframes de cada arquivo
dataframes = []

# Lê cada arquivo e adiciona ao dataframe
for arquivo in arquivos_1100:
    df = pd.read_csv(os.path.join(pasta_arquivos, arquivo), sep='|', header=None, 
                     names=['Cod_reg', 'Chv_doc', 'data', 'num_item', 'ind_oper', 
                             'cod_prod', 'cfop', 'qtd', 'icms_tot', 'vlr_confr', 'cod_legal'],
                    dtype=str)  # Se o arquivo estiver em formato CSV, ajuste os parâmetros conforme necessário
    dataframes.append(df)

# Concatena todos os dataframes em um único dataframe
df_final = pd.concat(dataframes, ignore_index=True)
df_final

,Cod_reg,Chv_doc,data,num_item,ind_oper,cod_prod,cfop,qtd,icms_tot,vlr_confr,cod_legal
0,1100,35200171605265039297590006031131297186579609,01012020,2,1,100388,5405,"1,000",NaN,"0,63",1
1,1100,35200171605265039297590006031131301828748473,03012020,3,1,100388,5405,"1,000",NaN,"0,63",1
2,1100,35200171605265039297590005812820461046577284,03012020,1,1,100388,5405,"1,000",NaN,"0,63",1
3,1100,35200127849963000110550010006604811006584101,04012020,67,0,100388,1403,"1,000","1,34",NaN,NaN
4,1100,35200171605265039297590006031131310043461524,06012020,2,1,100388,5405,"1,000",NaN,"0,63",1
...,...,...,...,...,...,...,...,...,...,...,...
493477,1100,35201271605265039297590007086090676940106572,23122020,3,1,99234,5405,"1,000",NaN,"1,12",1
493478,1100,35201271605265039297590005812821041405820710,24122020,5,1,99234,5405,"1,000",NaN,"1,12",1
493479,1100,35201227849963000110550010010262221010226748,29122020,19,0,99234,1403,"1,000","2,62",NaN,NaN
493480,1100,35201271605265039297590007086090696680045131,31122020,4,1,99234,5405,"1,000",NaN,"1,12",1


In [6]:
df_1100_icms_tot = df_final[['Chv_doc', 'num_item', 'icms_tot']]
ficha_3_icms_tot = ficha_3[['CHV_DOC', 'NUM_ITEM', 'ICMS_TOT_PCAT']]

ficha_3_icms_tot['NUM_ITEM'] = ficha_3_icms_tot['NUM_ITEM'].astype(str)
df_1100_icms_tot['num_item'] = df_1100_icms_tot['num_item'].astype(str)

df_1100_icms_tot['Chv_item'] = df_1100_icms_tot['Chv_doc'] + '-' + df_1100_icms_tot['num_item']
ficha_3_icms_tot['CHV_ITEM'] = ficha_3_icms_tot['CHV_DOC'] + '-' + ficha_3_icms_tot['NUM_ITEM']

C:\Users\vinic\AppData\Local\Temp\ipykernel_2896\1545256236.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ficha_3_icms_tot['NUM_ITEM'] = ficha_3_icms_tot['NUM_ITEM'].astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_2896\1545256236.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_icms_tot['num_item'] = df_1100_icms_tot['num_item'].astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_2896\1545256236.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [7]:
df_1100_icms_tot['icms_tot'] = df_1100_icms_tot['icms_tot'].str.replace(',', '.').astype(float)
df_1100_icms_tot

C:\Users\vinic\AppData\Local\Temp\ipykernel_2896\1930534112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_icms_tot['icms_tot'] = df_1100_icms_tot['icms_tot'].str.replace(',', '.').astype(float)


,Chv_doc,num_item,icms_tot,Chv_item
0,35200171605265039297590006031131297186579609,2,NaN,35200171605265039297590006031131297186579609-2
1,35200171605265039297590006031131301828748473,3,NaN,35200171605265039297590006031131301828748473-3
2,35200171605265039297590005812820461046577284,1,NaN,35200171605265039297590005812820461046577284-1
3,35200127849963000110550010006604811006584101,67,1.34,35200127849963000110550010006604811006584101-67
4,35200171605265039297590006031131310043461524,2,NaN,35200171605265039297590006031131310043461524-2
...,...,...,...,...
493477,35201271605265039297590007086090676940106572,3,NaN,35201271605265039297590007086090676940106572-3
493478,35201271605265039297590005812821041405820710,5,NaN,35201271605265039297590005812821041405820710-5
493479,35201227849963000110550010010262221010226748,19,2.62,35201227849963000110550010010262221010226748-19
493480,35201271605265039297590007086090696680045131,4,NaN,35201271605265039297590007086090696680045131-4


In [9]:
merged = df_1100_icms_tot.merge(ficha_3_icms_tot[['CHV_ITEM', 'ICMS_TOT_PCAT']],
                               left_on=['Chv_item'],
                               right_on=['CHV_ITEM'])

merged = merged.drop_duplicates()
merged

,Chv_doc,num_item,icms_tot,Chv_item,CHV_ITEM,ICMS_TOT_PCAT
0,35200171605265039297590006031131297186579609,2,NaN,35200171605265039297590006031131297186579609-2,35200171605265039297590006031131297186579609-2,NaN
1,35200171605265039297590006031131301828748473,3,NaN,35200171605265039297590006031131301828748473-3,35200171605265039297590006031131301828748473-3,NaN
2,35200171605265039297590005812820461046577284,1,NaN,35200171605265039297590005812820461046577284-1,35200171605265039297590005812820461046577284-1,NaN
3,35200127849963000110550010006604811006584101,67,1.34,35200127849963000110550010006604811006584101-67,35200127849963000110550010006604811006584101-67,1.335295
4,35200171605265039297590006031131310043461524,2,NaN,35200171605265039297590006031131310043461524-2,35200171605265039297590006031131310043461524-2,NaN
...,...,...,...,...,...,...
493477,35201271605265039297590007086090676940106572,3,NaN,35201271605265039297590007086090676940106572-3,35201271605265039297590007086090676940106572-3,NaN
493478,35201271605265039297590005812821041405820710,5,NaN,35201271605265039297590005812821041405820710-5,35201271605265039297590005812821041405820710-5,NaN
493479,35201227849963000110550010010262221010226748,19,2.62,35201227849963000110550010010262221010226748-19,35201227849963000110550010010262221010226748-19,2.621547
493480,35201271605265039297590007086090696680045131,4,NaN,35201271605265039297590007086090696680045131-4,35201271605265039297590007086090696680045131-4,NaN


In [10]:
merged['Diffs'] = np.abs(merged['icms_tot'] - merged['ICMS_TOT_PCAT'])
merged

,Chv_doc,num_item,icms_tot,Chv_item,CHV_ITEM,ICMS_TOT_PCAT,Diffs
0,35200171605265039297590006031131297186579609,2,NaN,35200171605265039297590006031131297186579609-2,35200171605265039297590006031131297186579609-2,NaN,NaN
1,35200171605265039297590006031131301828748473,3,NaN,35200171605265039297590006031131301828748473-3,35200171605265039297590006031131301828748473-3,NaN,NaN
2,35200171605265039297590005812820461046577284,1,NaN,35200171605265039297590005812820461046577284-1,35200171605265039297590005812820461046577284-1,NaN,NaN
3,35200127849963000110550010006604811006584101,67,1.34,35200127849963000110550010006604811006584101-67,35200127849963000110550010006604811006584101-67,1.335295,0.004705
4,35200171605265039297590006031131310043461524,2,NaN,35200171605265039297590006031131310043461524-2,35200171605265039297590006031131310043461524-2,NaN,NaN
...,...,...,...,...,...,...,...
493477,35201271605265039297590007086090676940106572,3,NaN,35201271605265039297590007086090676940106572-3,35201271605265039297590007086090676940106572-3,NaN,NaN
493478,35201271605265039297590005812821041405820710,5,NaN,35201271605265039297590005812821041405820710-5,35201271605265039297590005812821041405820710-5,NaN,NaN
493479,35201227849963000110550010010262221010226748,19,2.62,35201227849963000110550010010262221010226748-19,35201227849963000110550010010262221010226748-19,2.621547,0.001547
493480,35201271605265039297590007086090696680045131,4,NaN,35201271605265039297590007086090696680045131-4,35201271605265039297590007086090696680045131-4,NaN,NaN


In [11]:
if merged[merged['Diffs'] >= 0.1].shape[0] > 0:
    print('Há erro na geração do TXT em relação ao ICMS_TOT_PCAT, favor verificar')
    sys.exit()

In [12]:
df_final

,Cod_reg,Chv_doc,data,num_item,ind_oper,cod_prod,cfop,qtd,icms_tot,vlr_confr,cod_legal
0,1100,35200171605265039297590006031131297186579609,01012020,2,1,100388,5405,"1,000",NaN,"0,63",1
1,1100,35200171605265039297590006031131301828748473,03012020,3,1,100388,5405,"1,000",NaN,"0,63",1
2,1100,35200171605265039297590005812820461046577284,03012020,1,1,100388,5405,"1,000",NaN,"0,63",1
3,1100,35200127849963000110550010006604811006584101,04012020,67,0,100388,1403,"1,000","1,34",NaN,NaN
4,1100,35200171605265039297590006031131310043461524,06012020,2,1,100388,5405,"1,000",NaN,"0,63",1
...,...,...,...,...,...,...,...,...,...,...,...
493477,1100,35201271605265039297590007086090676940106572,23122020,3,1,99234,5405,"1,000",NaN,"1,12",1
493478,1100,35201271605265039297590005812821041405820710,24122020,5,1,99234,5405,"1,000",NaN,"1,12",1
493479,1100,35201227849963000110550010010262221010226748,29122020,19,0,99234,1403,"1,000","2,62",NaN,NaN
493480,1100,35201271605265039297590007086090696680045131,31122020,4,1,99234,5405,"1,000",NaN,"1,12",1


In [13]:
# Checagem de existência de ICMS_TOT

if df_final[(df_final['cfop'].isin(['1403', '1409', '1411', '5411'])) & (df_final['icms_tot'].isnull())].shape[0] > 0:
    print('Existem entradas, devoluções de entrada e devoluções de saída com ICMS_TOT nulo. Favor checar')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5409', '5405', '5403', '5401'])) & (df_final['icms_tot'].notnull())].shape[0] > 0:
    print('Existem saídas com campo ICMS_TOT preenchido. Favor checar')
    sys.exit()

In [23]:
# Checagem de Valor de confronto

if df_final[(df_final['cfop'].isin(['1403', '1409', '5411'])) & (df_final['vlr_confr'].notnull())].shape[0] > 0:
    print('Existem entradas e devoluções de entrada com Valor de Confronto não nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'] == '1') & (df_final['vlr_confr'].isnull())].shape[0] > 0:
    print('Existem sáidas e devoluções de saída com Valor de Confronto nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'] == '0') & (df_final['vlr_confr'].notnull())].shape[0] > 0:
    print('Existem saídas e devoluções de saída com Valor de Confronto não nulo quando Código Legal igual a 0. Favor, checar')
    sys.exit()

In [15]:
# Cheagem de código legal

if df_final[(df_final['cfop'].isin(['1403', '1409', '5411'])) & (df_final['cod_legal'].notnull())].shape[0] > 0:
    print('Existem entradas e devoluções de entrada com Código Legal não nulo. Favor, checar.')
    sys.exit()
    
if df_final[(df_final['cfop'].isin(['5405', '1411', '5409', '5403', '5401'])) & (df_final['cod_legal'].isnull())].shape[0] > 0:
    print('Existem sáidas e devoluções de saída com Código Legal nulo. Favor, checar.')
    sys.exit()

In [25]:
df_final[(df_final['cfop'] == '1411') & (df_final['cod_legal'] == '1')]['vlr_confr'].unique()

array(['0,01'], dtype=object)